# HDF5 export: Compare the size between double/simple precision
# -----------------------------------------------------------------

1st step : load dpf

In [ ]:
import tempfile
import os
tmpdir = tempfile.mkdtemp()

In [2]:
from ansys.dpf import core as dpf
from ansys.dpf.core import operators as ops
from ansys.dpf.core import examples

In [3]:
doc = ops.utility.html_doc()
doc.run()

2nd step : load the hdf5 plugin

In [4]:
dpf.load_library("Ans.Dpf.Hdf5.dll","hdf5")
from ansys.dpf.core import operators as ops

3rd step: read stresses, displacements, and the mesh from an rst file and export it to hdf5

In [5]:
transient = examples.download_transient_result()
model = dpf.Model(transient)

In [6]:
stress = model.results.stress()
displacement = model.results.displacement()
mesh = model.metadata.meshed_region

In [7]:
timeIds = list(range(1,model.metadata.time_freq_support.n_sets+1))

By default, we only read the last time step, a user can request one or several time steps using the time_scoping input. Here we will request all time steps

In [8]:
stress.inputs.time_scoping.connect(timeIds)
displacement.inputs.time_scoping.connect(timeIds)

In [9]:
h5op = ops.serialization.serialize_to_hdf5()

In [10]:
print(h5op)

DPF serialize_to_hdf5 Operator: 
  Serialize the inputs in an hdf5 format. 
  Inputs:
         file_path [string]: output file path with .h5 extension 
         export_floats [bool]: converts double to float to reduce file size (default is true) 
         export_flat_vectors [bool]: if true, vectors and matrices data are exported flat (x1,y1,z1,x2,y2,z2..) (default is false) 
         data (ellipsis) []: only the data set explicitly to export is exported 
  Run the operator to get its result



export to hdf5 with simple precision

In [11]:
h5op.inputs.file_path.connect(os.path.join(tmpdir,'c:/temp/dpf_float.h5'))

In [12]:
h5op.inputs.data1.connect(stress.outputs)
h5op.inputs.data2.connect(displacement.outputs)
h5op.inputs.data3.connect(mesh)

In [13]:
h5op.run()

export to hdf5 with double precision

In [14]:
h5op.inputs.export_floats.connect(False)

In [15]:
h5op.inputs.file_path.connect(os.path.join(tmpdir,'c:/temp/dpf_double.h5'))

In [16]:
h5op.run()

In [17]:
print(f"size with float precision: {os.stat(os.path.join(tmpdir,'c:/temp/dpf_float.h5')).st_size} \n size with double precision: {os.stat(os.path.join(tmpdir,'c:/temp/dpf_double.h5')).st_size}")

size with float precision: 6702616 
 size with double precision: 13111816


The batch file cannot be found.

